# 04 — Baseline: ResNet18 + Augmentation Leve




Arquivos utilizados:
- data/metadata/train_split.csv
- data/metadata/val_split.csv

Configuração deste experimento:

Modelo: ResNet18  
Augmentation: light  
Class Weight: False  
Epochs: 10  
Batch Size: 32  
Learning Rate: 1e-4  

Objetivo:

Estabelecer o desempenho base do projeto.
As métricas serão salvas em:

- models/resnet18_light_noCW.pt
- outputs/metrics/resnet18_light_noCW.pkl


In [1]:
import os

NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

print("Project root:", PROJECT_ROOT)


Project root: c:\projects\xray-project


In [2]:
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from dataset import XRayDataset
from transforms import get_transforms
from model import get_model
from train_utils import train_model
from utils import set_seed, create_directories, save_metrics
from torch.utils.data import DataLoader


## Controle de Reprodutibilidade

Fixou-se a semente global para garantir consistência entre execuções.


In [3]:
set_seed(42)


## Preparação de Diretórios

Criar automaticamente:

- models/
- outputs/metrics/


In [4]:
models_dir, metrics_dir, _ = create_directories(PROJECT_ROOT)

print("Models:", models_dir)
print("Metrics:", metrics_dir)


Models: c:\projects\xray-project\models
Metrics: c:\projects\xray-project\outputs\metrics


## Carregamento dos Dados

Utilizar os splits congelados para garantir comparação justa entre experimentos.


In [5]:
metadata_dir = os.path.join(PROJECT_ROOT, "data", "metadata")

train_df = pd.read_csv(os.path.join(metadata_dir, "train_split.csv"))
val_df = pd.read_csv(os.path.join(metadata_dir, "val_split.csv"))

print("Treino:", len(train_df))
print("Validação:", len(val_df))


Treino: 4093
Validação: 1139


## Configuração do Baseline

Modelo: ResNet18  
Augmentation: leve  
Class Weight: não utilizado  
Épocas: 10  
Batch Size: 32  
Learning Rate: 1e-4


In [6]:
model_name = "resnet18"
augmentation = "light"
use_class_weight = False

batch_size = 16
epochs = 10
learning_rate = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)


Dispositivo: cpu


## Datasets e DataLoaders

Transformações definidas em src/transforms.py  
Dataset definido em src/dataset.py


In [7]:
train_transform, val_transform = get_transforms(
    img_size=224,
    augmentation=augmentation
)

train_dataset = XRayDataset(train_df, transform=train_transform)
val_dataset = XRayDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## Instanciação do Modelo

Modelo definido em src/model.py


In [8]:
model, target_layer = get_model(model_name=model_name)
model = model.to(device)


c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Função de Perda e Otimizador

Neste baseline não utiliza class weight.


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Padrão de Nomeação

Formato:

modelo_augmentation_classweight


In [ ]:
model_filename = f"{model_name}_{augmentation}_noCW.pt"
metrics_filename = f"{model_name}_{augmentation}_noCW.npz"

model_save_path = os.path.join(models_dir, model_filename)
metrics_save_path = os.path.join(metrics_dir, metrics_filename)
# Geração de .npz no baseline mas outro modelos irão gerar .pkl que será o padrão

## Treinamento

O loop de treinamento está implementado em src/train_utils.py.

A cada época são exibidos:
- Loss de treino
- Loss de validação
- ROC-AUC de validação

O melhor modelo é salvo automaticamente.


In [11]:
history, best_auc = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs,
    model_save_path
)



Epoch 1/10
Train Loss: 0.1282
Val Loss:   0.0580
Val AUC:    0.9980
Val F1:     0.9856
Val Recall: 0.9782
Val Prec.:  0.9930
→ Novo melhor modelo salvo.



Epoch 2/10
Train Loss: 0.0741
Val Loss:   0.1286
Val AUC:    0.9980
Val F1:     0.9663
Val Recall: 0.9369
Val Prec.:  0.9976
→ Novo melhor modelo salvo.



Epoch 3/10
Train Loss: 0.0484
Val Loss:   0.0515
Val AUC:    0.9986
Val F1:     0.9879
Val Recall: 0.9828
Val Prec.:  0.9930
→ Novo melhor modelo salvo.



Epoch 4/10
Train Loss: 0.0442
Val Loss:   0.0517
Val AUC:    0.9988
Val F1:     0.9892
Val Recall: 1.0000
Val Prec.:  0.9787
→ Novo melhor modelo salvo.



Epoch 5/10
Train Loss: 0.0379
Val Loss:   0.0441
Val AUC:    0.9990
Val F1:     0.9885
Val Recall: 0.9817
Val Prec.:  0.9953
→ Novo melhor modelo salvo.



Epoch 6/10
Train Loss: 0.0318
Val Loss:   0.0423
Val AUC:    0.9989
Val F1:     0.9885
Val Recall: 0.9828
Val Prec.:  0.9942



Epoch 7/10
Train Loss: 0.0346
Val Loss:   0.0652
Val AUC:    0.9988
Val F1:     0.9826
Val Recall: 0.9690
Val Prec.:  0.9965



Epoch 8/10
Train Loss: 0.0231
Val Loss:   0.0544
Val AUC:    0.9985
Val F1:     0.9885
Val Recall: 0.9817
Val Prec.:  0.9953



Epoch 9/10
Train Loss: 0.0161
Val Loss:   0.0444
Val AUC:    0.9986
Val F1:     0.9897
Val Recall: 0.9885
Val Prec.:  0.9908



Epoch 10/10
Train Loss: 0.0119
Val Loss:   0.1030
Val AUC:    0.9990
Val F1:     0.9784
Val Recall: 0.9599
Val Prec.:  0.9976

Melhor AUC obtida: 0.9990293096931588


## Salvamento das Métricas

As métricas serão utilizadas no notebook final
de comparação entre hipóteses.


In [13]:
metrics_filename = f"{model_name}_{augmentation}_{'CW' if use_class_weight else 'noCW'}.pkl"
metrics_save_path = os.path.join(metrics_dir, metrics_filename)

metrics_dict = {
    "model_name": model_name,
    "augmentation": augmentation,
    "class_weight": use_class_weight,
    "epochs": epochs,
    "history": history,
    "best_auc": best_auc
}

save_metrics(metrics_dict, metrics_save_path)

print("Modelo salvo em:", model_save_path)
print("Métricas salvas em:", metrics_save_path)
print("Melhor AUC:", best_auc)

Modelo salvo em: c:\projects\xray-project\models\resnet18_light_noCW.pt
Métricas salvas em: c:\projects\xray-project\outputs\metrics\resnet18_light_noCW.pkl
Melhor AUC: 0.9990293096931588
